# **Technical Prerequisites**

This document is a Jupyter Notebook. A Jupyter Notebook is a document that contains a mix of static text (like the one you are reading) and executable code (like the one you see if you scroll down). 

The static text can be enriched with mathematical formulas and media such as images or videos, making notebooks a powerful tool to write and present code with explanations. \
The code is usually (and in our case, it is) Python, but you might be interested in knowing that it can be R and a wealth of other programming languages.

The figure below illustrates the basic anatomy of a Jupyter Notebook as I see it on my laptop:

<p align="center">
<img src="./files/jupyter-notebook-anatomy.png" width="1100"/>
</p>

**Text cells:**
- To write text in a text cell, double-click on it or press `Enter` after a single click. This will put the text cell into edit mode
- Once you are done editing, press `Ctrl + Enter` to render your text
- You can switch between edit and rendered mode as many times as you like

**Code cells:**
- To write code in a code cell, just click on it and start typing
- To run a code cell, click on it to select it, then press `Ctrl + Enter`

---

Feel free to delete this cell once you are done learning its content. To do so, you can select it and press `D` twice rapidly. \
If you regret deleting it, you can alway press `Ctrl + Z` to restore it (but it only works if you do it promptly).

# **Introduction**

Students write text here, explaining:
 - What it means to preprocess data and what is a preprocessing pipeline
 - What are the main artefacts that can be found in the TMS-EEG signal and their causes
 - **Optional**: a personal comment or opinion on the problems posed by preprocessing, based on what you heard in class during both theoretical lessons and the first _briefing_

In [1]:
from pathlib import Path                            # to build a bridge between Python and the filesystem 

import numpy as np                                  # to perform array operations
import matplotlib                                   # this and the following to draw plots
import matplotlib.pyplot as plt
matplotlib.use('Qt5Agg')                            # to make plots interactive

import mne                                          # to read and manipulate EEG data

from scripts import utils                           # custom functions to shorten the code in this notebook

# **1. Basic Preprocessing (Assignment 1, Deadline 26/11/2025)**

Students write text here, listing the operations that we categorise as "basic preprocessing".

If you cannot remember what the steps are, look at the code in the cells below and you will recall. 

## **1.1. Students Write the Name of This Step**

**Optional but welcome:** students write here any considerations about this step.

In [ ]:
data_dir = Path("data")
subject_and_condition = "S02C1_M1"

file_of_interest = list(data_dir.glob(pattern=f"{subject_and_condition}.vhdr"))
print(file_of_interest, "|", type(file_of_interest), "|", file_of_interest[0])

In [ ]:
eeg_data = mne.io.read_raw_brainvision(vhdr_fname=file_of_interest[0])

## **1.2. Students Replace This Sentence With the Name of This Step**

**Compulsory:** students write here the functional significance of this step (that is, why we do it).

In [ ]:
channels_to_drop = []

for channel_name in eeg_data.ch_names:
    try:
        int(channel_name[-1])
    except ValueError:
        if channel_name[-1] != "z": 
            channels_to_drop.append(channel_name)
print(f"Channels to drop: {channels_to_drop}")

eeg_data = eeg_data.drop_channels(ch_names=channels_to_drop)

## **1.3. Students Replace This Sentence With the Name of This Step**

**Compulsory:** students write here the functional significance of this step (that is, why we do it).

In [ ]:
easycap_m1_montage = mne.channels.make_standard_montage(kind="easycap-M1")
easycap_m1_montage.plot()
eeg_data.set_montage(montage=easycap_m1_montage,
                     on_missing="ignore")

**Compulsory:** here, students describe and comment the plot generated by the code below. In particular, I would like to read:

- What each row represents
- What are the $x$ and $y$ of each row, and their units of measurement
- A description of the artefacts you see, and their likely causes

In [ ]:
eeg_data.plot()

**Compulsory:** here, students define event markers and explain why they are important for EEG data analysis

**Optional:** any comments that go beyond a mere definition are welcome. For example, this could be comments about the structure of event arrays and what it tells us about the nature of events

In [ ]:
events_from_annotations, events_dict = mne.events_from_annotations(raw=eeg_data)
events_from_annotations = events_from_annotations[2:]

## **1.4. Students Replace This Sentence With the Name of This Step**

**Compulsory:** students write here the functional significance of this step (that is, why we do it).

In [ ]:
pre_interpolation_epochs = mne.Epochs(raw=eeg_data,
                                      events=events_from_annotations,
                                      tmin=-1.1,
                                      tmax=0.5,
                                      baseline=None)
pre_interpolation_epochs_tep = pre_interpolation_epochs.average()
pre_interpolation_epochs_tep.plot();

In [ ]:
post_interpolation_eeg = mne.io.read_raw(fname="data/post_2_5_interpolation_eeg.fif",
                                         preload=True)
post_interpolation_epochs = mne.Epochs(raw=post_interpolation_eeg,
                                       events=events_from_annotations,
                                       tmin=-1.1,
                                       tmax=0.5,
                                       baseline=None)
post_interpolation_tep = post_interpolation_epochs.average()
post_interpolation_tep.plot();

## **1.5. Students Replace This Sentence With the Name of This Step**

**Compulsory:** students write here the functional significance of this step (that is, why we do it) and potential caveats (that is, one-two things that can happen if you do this step unproperly).

In [ ]:
post_filtering_eeg = mne.filter.filter_data(data=post_interpolation_eeg.get_data(),
                                            sfreq=post_interpolation_eeg.info["sfreq"],
                                            l_freq=0.1,
                                            h_freq=None,
                                            method="iir",
                                            iir_params=None,
                                            copy=True,
                                            phase="zero")
post_filtering_eeg = mne.io.RawArray(data=post_filtering_eeg,
                                     info=post_interpolation_eeg.info)
post_filtering_epochs = mne.Epochs(raw=post_filtering_eeg,
                                   events=events_from_annotations,
                                   tmin=-1.1,
                                   tmax=0.5,
                                   baseline=None)
post_filtering_tep = post_filtering_epochs.average()
post_filtering_tep.plot();

# **2. Independent Component Analysis (ICA) (Assignment 2, Deadline 05/12/2025)**

## **2.1. Rationale**

**Compulsory:** students write here the functional significance of this step (that is, why we do it) and expand on its rationale, devoting particular attention to discussing the independence assumption and why its application to brain data is debatable.

## **2.2. Fitting**

In [ ]:
ica = mne.preprocessing.ICA(random_state=0)
ica.fit(post_filtering_epochs)

## **2.3. Components Selection**

**Compulsory:** students select components whose most likely source are eye movements and justify their choices here, including a picture of the selected component(s).

To help you tell components apart, you can refer to [this](https://labeling.ucsd.edu/tutorial/labels) tutorial by the developers of ICLabel: an algorithm to automatically classify independent components that came out of the Swartz Center for Computational Neuroscience, University of California San Diego ([Pion-Tonachini et al., 2019](https://www.sciencedirect.com/science/article/pii/S1053811919304185)).

To include a picture of the selected component(s), just take a screenshot, save it somewhere and change the path below (`files/sample-ic.png`) with the path to your screenshot. 

<p align="center">
<img src="./files/sample-ic.png" width="1000"/>
</p>

Selecting no components is OK but the choice must be justified in writing.

In [ ]:
ica.plot_components(inst=post_filtering_epochs, picks=range(30))

In [ ]:
ica.plot_sources(inst=post_filtering_epochs)

In [ ]:
components_to_reject = utils.select_items(item_type="ica")

In [ ]:
ica.exclude = components_to_reject
ica.apply(post_filtering_epochs.load_data())
post_ica_epochs = post_filtering_epochs
post_ica_tep = post_ica_epochs.average()
post_ica_tep.plot();

# **3. Manual Artifact Rejection (Assignment 3, Deadline 12/12/2025)**

## **3.1. Rationale**

**Compulsory:** students write here the functional significance of this step (that is, why we do it) and expand on the criteria to follow for its execution. 

## **3.2. Execution**

In [ ]:
post_ica_epochs.plot()

# **4. Computing & Assessing a TEP (Assignment 4, Deadline 19/12/2025)**

## **4.1. Preparatory Steps**

**Compulsory:** for each preparatory step, students describe its functional significance (that is, why we do it) above the corresponding code cell

In [ ]:
low_pass_filtered_data = mne.filter.filter_data(data=post_ica_epochs.get_data(),
                                                sfreq=post_ica_epochs.info["sfreq"],
                                                l_freq=None,
                                                h_freq=70.0,
                                                method="iir",
                                                iir_params=None,
                                                copy=True,
                                                phase="zero")
low_pass_filtered_epochs = mne.EpochsArray(data=low_pass_filtered_data,
                                           info=post_ica_epochs.info,
                                           baseline=None)
low_pass_filtered_tep = low_pass_filtered_epochs.average()
low_pass_filtered_tep.plot();

In [ ]:
rereferenced_epochs, reference = mne.set_eeg_reference(inst=low_pass_filtered_epochs,
                                                       ref_channels="average") 

In [ ]:
TIME_OF_EVENT = 1.1                             # seconds
baseline_window_start = TIME_OF_EVENT - 0.100   # seconds
baseline_window_end = TIME_OF_EVENT - 0.002     # seconds

In [ ]:
baseline_corrected_epochs = rereferenced_epochs.apply_baseline(baseline=(baseline_window_start, baseline_window_end))
baseline_corrected_tep = baseline_corrected_epochs.average()
baseline_corrected_tep.plot();

In [ ]:
final_epochs = baseline_corrected_epochs.crop(tmin=1.0,
                                              tmax=1.6,
                                              include_tmax=True)
final_tep = final_epochs.average()
final_tep.plot();

## **4.2. Computation of the TEP** 

# **Closing Remarks**

**Optional:** Student can write here any comment or consideration that goes beyond the minimum required, for example:
- "I think filtering is `{good/bad}` because..."
- "I think these data were particularly `{dirty/clean}` because..."
- "I think that this thing is an issue because..."
- Anything else you might feel like saying about the preprocessing you have done

Do not be afraid to make such comments: there is no right or wrong and they do not count for the vote. At most, a great comment could make you a _cum laude_ candidate, but I really just want to see if you have a personal opinion on the subject.